In [1]:
#@title Colab setup

import os, sys, subprocess
if "google.colab" in sys.modules:
  cmd = "pip install --upgrade biocircuits bokeh-catplot watermark blackcellmagic"
  process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  stdout, stderr = process.communicate()
# ------

import scipy.integrate
import bokeh.io
import numpy as np

import bokeh.io
import bokeh.plotting
from bokeh.models import LinearColorMapper, ColorBar

bokeh.io.output_notebook()

In [2]:
#@title Auxiliary functions

# Function to do the 2D projection of decision boundaries with normalization
def contourf(p, x, y, z, title=None, palette="Spectral11", normal = True):
    """Make a filled contour plot given x, y, z data given in 2D arrays."""

    # Normalize the z values
    if normal:
      z_min = z.min()
      z_max = z.max()
      z_normalized = (z - z_min) / (z_max - z_min)  # Normalize to range [0, 1]
    else:
      z_normalized = z

    # Add zero padding at the boundaries for better visualization
    N = z_normalized.shape[1]
    z0 = np.c_[z_normalized, np.zeros(N)]
    z0[-1, -1] = 1.  # Ensure the padding contains a value within range

    # Plot the normalized values
    p.image(
        image=[z0],
        x=x.min(),
        y=y.min(),
        dw=(x.max() - x.min()) * (1 + 1 / N),
        dh=x.max() - x.min(),
        palette=palette,
        alpha=0.8,
    )

    # Color mapping based on the normalized z0 values
    color = LinearColorMapper(palette=palette, low=z0.min(), high=z0.max())
    cb = ColorBar(color_mapper=color, location=(0, 0), width=10)
    p.add_layout(cb, 'right')

    return ()

## Learning as a random search for the adequate weights

The following code implements the example described in Figure 3 to illustrate how weight tuning enables a different decision boundary.

In [7]:
def nonlinear_classifier(X, t, x1, x2, w_hidden, w_output, d, g):
    # Unpacking the weights
    w1_0, w1_1, w1_2 = w_hidden[0]  # First node weights (hidden node 1)
    w2_0, w2_1, w2_2 = w_hidden[1]  # Second node weights (hidden node 2)
    w3_0, w3_1, w3_2 = w_output     # Output node weights (node 3)

    # Unpacking the state variables
    z1_n1, z2_n1, z1_n2, z2_n2, z1, z2 = X

    # Solve for the first hidden node (Node 1)
    dz1_n1_dt = w1_0 + x2*w1_2 - d*z1_n1 - g*z1_n1*z2_n1  # Modify z1_n1 with x2
    dz2_n1_dt = x1*w1_1 - d*z2_n1 - g*z1_n1*z2_n1          # Modify z2_n1 with x1

    # Solve for the second hidden node (Node 2)
    dz1_n2_dt = w2_0 + x1*w2_1 - d*z1_n2 - g*z1_n2*z2_n2  # Modify z1_n2 with x1
    dz2_n2_dt = x2*w2_2 - d*z2_n2 - g*z1_n2*z2_n2          # Modify z2_n2 with x2

    # Solve for the output node (Node 3)
    a = z1_n1*w3_1 + z1_n2*w3_2  # Output node: combination of hidden nodes
    b = w3_0
    dz1_dt = a - d*z1 - g*z1*z2  # Modify z1 in the output node
    dz2_dt = b - d*z2 - g*z1*z2  # Modify z2 in the output node

    return [dz1_n1_dt, dz2_n1_dt, dz1_n2_dt, dz2_n2_dt, dz1_dt, dz2_dt]

In [11]:
#@title In abundance of nutrients ($x_1$ and $x_2$)

# First Node's weights
w1_0 = 0.2
w1_1 = 1.0
w1_2 = 0.8

# Second node's weights
w2_0 = 0.4
w2_1 = 1.0
w2_2 = 1.0

# Third node (output)
w3_0 = 0.1
w3_1 = 1.0
w3_2 = 1.0

# Set up figures
fig_size = [300, 225]
plots = []
for i in range(3):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1]))
    plots[i].axis.major_label_text_font_size = '10pt'

# Simulation parameters
t = np.linspace(0,10,100) # 10 hours
g = 100 # sequestration rate
d = 1 # degradation rate

# Input space
N = 30 # resolution
x1 = np.linspace(0,1,N) # [uM]
x2 = np.linspace(0,1,N) # [uM]

# Seting up simulations
output1 = np.zeros((N,N))
output2 = np.zeros((N,N))
output3 = np.zeros((N,N))

# Organizing weights into arrays
w_hidden = np.array([
    [w1_0, w1_1, w1_2],  # First hidden node (w1_0, w1_1, w1_2)
    [w2_0, w2_1, w2_2]   # Second hidden node (w2_0, w2_1, w2_2)
])

w_output = np.array([w3_0, w3_1, w3_2])  # Output node weights (w3_0, w3_1, w3_2)

# Solving ODEs
for i, x1i in enumerate(x1):
    for j, x2j in enumerate(x2):
        # Solve the full system at once
        y_sol = scipy.integrate.odeint(nonlinear_classifier,
                                       [0., 0., 0., 0., 0., 0.], t,
                                       args=(x1i, x2j, w_hidden, w_output, d, g))

        # We are interested in steady-state values
        output1[j, i] = y_sol[-1, 0]  # Output of first node
        output2[j, i] = y_sol[-1, 2]  # Output of second node
        output3[j, i] = y_sol[-1, 4]  # Output of final node

# Visualizing decision boundary
palette = bokeh.palettes.Greys[9][::-1]
contourf(plots[0], x1, x2, output1, palette=palette, normal=True)
contourf(plots[1], x1, x2, output2, palette=palette, normal=True)
contourf(plots[2], x1, x2, output3, palette=palette, normal=True)

# Tidy up
for i in range(3):
    plots[i].x_range = bokeh.models.Range1d(x1[0], x1[-1])
    plots[i].y_range = bokeh.models.Range1d(x2[0], x2[-1])
    plots[i].output_backend = 'svg'

plots[0].title.text = '(Hidden) Node 1. Linear'
plots[1].title.text = ' (Hidden) Node 2. Linear'
plots[2].title.text = '(Output) Node 3. Nonlinear'

bokeh.io.show(bokeh.layouts.row(plots))

In [13]:
#@title In the scarcity of nutrients (evolved)

# First Node's weights
w1_0 = 0.2
w1_1 = 0.4 # decreased
w1_2 = 1.0 # incremented

# Second node's weights
w2_0 = 0.1 # decreased
w2_1 = 1.0
w2_2 = 0.5 # decreased

# Third node (output)
w3_0 = 0.1
w3_1 = 1.0
w3_2 = 1.0

# Set up figures
fig_size = [300, 225]
plots = []
for i in range(3):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1]))
    plots[i].axis.major_label_text_font_size = '10pt'

# Simulation parameters
t = np.linspace(0,10,100) # 10 hours
g = 100 # sequestration rate
d = 1 # degradation rate

# Input space
N = 30 # resolution
x1 = np.linspace(0,1,N) # [uM]
x2 = np.linspace(0,1,N) # [uM]

# Seting up simulations
output1 = np.zeros((N,N))
output2 = np.zeros((N,N))
output3 = np.zeros((N,N))

# Organizing weights into arrays
w_hidden = np.array([
    [w1_0, w1_1, w1_2],  # First hidden node (w1_0, w1_1, w1_2)
    [w2_0, w2_1, w2_2]   # Second hidden node (w2_0, w2_1, w2_2)
])

w_output = np.array([w3_0, w3_1, w3_2])  # Output node weights (w3_0, w3_1, w3_2)

# Solving ODEs
for i, x1i in enumerate(x1):
    for j, x2j in enumerate(x2):
        # Solve the full system at once
        y_sol = scipy.integrate.odeint(nonlinear_classifier,
                                       [0., 0., 0., 0., 0., 0.], t,
                                       args=(x1i, x2j, w_hidden, w_output, d, g))

        # We are interested in steady-state values
        output1[j, i] = y_sol[-1, 0]  # Output of first node
        output2[j, i] = y_sol[-1, 2]  # Output of second node
        output3[j, i] = y_sol[-1, 4]  # Output of final node

# Visualizing decision boundary
palette = bokeh.palettes.Greys[9][::-1]
contourf(plots[0], x1, x2, output1, palette=palette, normal=True)
contourf(plots[1], x1, x2, output2, palette=palette, normal=True)
contourf(plots[2], x1, x2, output3, palette=palette, normal=True)

# Tidy up
for i in range(3):
    plots[i].x_range = bokeh.models.Range1d(x1[0], x1[-1])
    plots[i].y_range = bokeh.models.Range1d(x2[0], x2[-1])
    plots[i].output_backend = 'svg'

plots[0].title.text = '(Hidden) Node 1. Linear'
plots[1].title.text = ' (Hidden) Node 2. Linear'
plots[2].title.text = '(Output) Node 3. Nonlinear'

bokeh.io.show(bokeh.layouts.row(plots))